<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=c92fb39d86b1556585a0cf71b90b26d1b049312c1ae8f1e8ec1b216b2ed14346
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

Mounted at /content/drive


In [4]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [5]:
# conviction distribution
mypf['Conviction'].value_counts()

M    79
L    65
H    38
Name: Conviction, dtype: int64

In [6]:
# star stocks
mypf['StarStock'].value_counts()

0.0    93
1.0    83
Name: StarStock, dtype: int64

In [7]:
# v40/v40n stocks
mypf['Criteria'].value_counts()

V40N    39
V40     38
2T3Y    10
SS       7
V200     6
TBD      2
Name: Criteria, dtype: int64

In [8]:
# latest quarter
mypf['LatestQtr'].value_counts()

1.0    165
0.0     16
Name: LatestQtr, dtype: int64

In [9]:
# star + v40/v40n stocks
mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]['StarStock'].value_counts()

1.0    41
0.0    36
Name: StarStock, dtype: int64

In [10]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    143
1     39
Name: InPortfolio, dtype: int64

In [11]:
# star stocks in portfolio
mypf[(mypf['InPortfolio'] == 1)]['StarStock'].value_counts()

1.0    23
0.0    13
Name: StarStock, dtype: int64

In [12]:
# criteria of portfolio stocks
mypf[(mypf['InPortfolio'] == 1)]['Criteria'].value_counts()

V40     12
2T3Y    10
V40N     8
SS       6
TBD      2
V200     1
Name: Criteria, dtype: int64

In [13]:
# strategies in portfolio stocks
mypf[(mypf['InPortfolio'] == 1)]['Strategy'].value_counts()

ATH     24
2T3Y    14
Name: Strategy, dtype: int64

In [14]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [15]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000

  return current_pe


In [16]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)

    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)
    #stock_df['RSI'] = round(stock_df['RSI'],0)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
    tmp['Curr_PE'] = current_pe
    #tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0]
    #tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
    tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
    tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

In [17]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-05-19 00:00:00+05:30,18203.0,^NSEI,2.32,0.54,0.48,18826.0,7610.0,5.55,3.0
2023-05-26 00:00:00+05:30,18499.0,^NSEI,3.79,0.41,0.42,18826.0,7610.0,2.92,2.0
2023-06-02 00:00:00+05:30,18534.0,^NSEI,3.83,0.34,0.84,18826.0,7610.0,2.60,2.0
2023-06-09 00:00:00+05:30,18563.0,^NSEI,3.87,0.27,0.87,18826.0,7610.0,2.34,1.0
2023-06-16 00:00:00+05:30,18826.0,^NSEI,5.18,0.27,0.86,18826.0,7610.0,0.00,0.0


In [18]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [19]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [20]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50', 'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 39


,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,1009.0,-35.69,-4.46,-2.39,96.64,328.0,1225.87,45.2,20.20,M,55.0,1.0,0.0,2T3Y,2T3Y
VALIANTORG.NS,558.0,-2.25,0.00,2.69,87.90,232.0,542.52,24.0,17.80,M,26.0,0.0,0.0,2T3Y,2T3Y
VAIBHAVGBL.NS,308.0,-3.91,0.00,0.97,76.60,214.0,317.48,35.4,48.50,M,-37.0,1.0,0.0,2T3Y,2T3Y
LUXIND.NS,1510.0,-2.43,-0.97,2.27,82.12,201.0,1493.72,21.4,33.00,M,-54.0,1.0,0.0,V40N,2T3Y
BANDHANBNK.NS,254.0,4.51,-0.41,2.51,82.43,185.0,253.98,NaN,1000.00,L,NaN,1.0,1.0,2T3Y,2T3Y
NIACL.NS,123.0,16.08,1.89,1.77,81.17,158.0,117.07,NaN,19.40,M,NaN,1.0,0.0,TBD,ATH
AAVAS.NS,1354.0,-26.23,-2.29,-1.83,71.93,136.0,1390.60,NaN,24.90,M,NaN,1.0,1.0,2T3Y,2T3Y
TEAMLEASE.NS,2387.0,-6.50,-2.04,0.59,76.14,121.0,2279.93,NaN,36.00,M,NaN,1.0,1.0,V40N,2T3Y
INDIGOPNTS.NS,1418.0,8.75,-0.54,3.18,79.61,119.0,1407.73,104.9,51.20,M,51.0,1.0,1.0,V40N,2T3Y


In [21]:
# top stocks from portfolio for SIP
tmp_df[tmp_df['Dev%_200'] < 0].sort_values(by = 'Upside%ATH', ascending=True)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
TCS.NS,3175.0,-0.82,-0.03,0.25,29.42,21.0,3231.00,28.1,27.6,H,2.0,1.0,1.0,V40,NaN
AMBUJACEM.NS,461.0,-0.02,0.65,2.18,27.43,28.0,424.02,23.9,32.7,L,-37.0,0.0,0.0,TBD,ATH
BATAINDIA.NS,1616.0,-0.30,-0.93,1.32,39.67,34.0,1523.62,126.0,64.3,H,49.0,1.0,0.0,V40,ATH
PAGEIND.NS,38568.0,-10.73,-1.24,0.15,40.00,39.0,39325.03,97.7,75.3,H,23.0,1.0,1.0,V40,ATH
BAYERCROP.NS,4250.0,-5.27,-1.00,0.48,54.14,41.0,4185.24,NaN,388.0,M,NaN,1.0,1.0,V40N,ATH
INFY.NS,1292.0,-8.59,-0.85,-0.55,40.64,43.0,1263.00,29.0,22.4,H,23.0,1.0,1.0,V40,ATH
FINEORG.NS,4917.0,-9.39,-1.20,1.43,35.27,46.0,4697.43,NaN,24.4,M,NaN,1.0,1.0,V40N,ATH
GILLETTE.NS,4540.0,-5.96,-0.79,0.54,89.41,66.0,4475.85,61.2,44.6,M,27.0,0.0,0.0,V40,ATH
JUBLFOOD.NS,498.0,-3.08,-0.97,1.29,58.11,81.0,471.56,90.5,93.1,H,-3.0,1.0,1.0,SS,ATH


In [22]:
# top 10 from portfolio for SIP
tmp_df[tmp_df['Dev%_200'] < 0].sort_values(by = 'Upside%ATH', ascending=False).head(10)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,1009.0,-35.69,-4.46,-2.39,96.64,328.0,1225.87,45.2,20.2,M,55.0,1.0,0.0,2T3Y,2T3Y
VALIANTORG.NS,558.0,-2.25,0.00,2.69,87.90,232.0,542.52,24.0,17.8,M,26.0,0.0,0.0,2T3Y,2T3Y
VAIBHAVGBL.NS,308.0,-3.91,0.00,0.97,76.60,214.0,317.48,35.4,48.5,M,-37.0,1.0,0.0,2T3Y,2T3Y
LUXIND.NS,1510.0,-2.43,-0.97,2.27,82.12,201.0,1493.72,21.4,33.0,M,-54.0,1.0,0.0,V40N,2T3Y
BANDHANBNK.NS,254.0,4.51,-0.41,2.51,82.43,185.0,253.98,NaN,1000.0,L,NaN,1.0,1.0,2T3Y,2T3Y
NIACL.NS,123.0,16.08,1.89,1.77,81.17,158.0,117.07,NaN,19.4,M,NaN,1.0,0.0,TBD,ATH
AAVAS.NS,1354.0,-26.23,-2.29,-1.83,71.93,136.0,1390.60,NaN,24.9,M,NaN,1.0,1.0,2T3Y,2T3Y


In [23]:
# top 5 from portfolio to book profit
tmp_df.sort_values(by = 'Upside%ATH', ascending=True).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ASIANPAINT.NS,3317.0,9.36,-0.30,1.69,9.21,7.0,3091.34,87.9,77.0,H,12.0,1.0,1.0,V40,ATH
PIDILITIND.NS,2698.0,6.00,-0.04,1.35,10.18,8.0,2504.06,94.6,108.0,H,-14.0,1.0,1.0,V40,ATH
RELIANCE.NS,2577.0,4.60,-0.24,1.07,12.04,9.0,2467.15,29.1,26.1,H,10.0,1.0,1.0,V40,ATH
BAJFINANCE.NS,7203.0,9.35,-0.14,2.32,11.48,10.0,6602.15,52.6,37.9,H,28.0,1.0,1.0,V40,ATH
TCS.NS,3175.0,-0.82,-0.03,0.25,29.42,21.0,3231.00,28.1,27.6,H,2.0,1.0,1.0,V40,NaN
BAJAJFINSV.NS,1509.0,0.37,-0.53,1.71,26.57,26.0,1454.93,39.2,37.4,M,5.0,1.0,1.0,V40,ATH
AMBUJACEM.NS,461.0,-0.02,0.65,2.18,27.43,28.0,424.02,23.9,32.7,L,-37.0,0.0,0.0,TBD,ATH


In [24]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [25]:
# star + v40/v40n stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 26)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
SUNTV.NS,445.0,-4.95,-0.21,0.92,755.0,249.0,61.26,70.0,M,1.0,1.0,V40N,NaN
RAJESHEXPO.NS,621.0,-7.38,0.00,-0.34,949.0,440.0,64.44,53.0,M,1.0,1.0,V40N,NaN
ASTRAZEN.NS,3643.0,10.61,0.73,1.03,4650.0,1316.0,30.20,28.0,M,1.0,1.0,V40N,NaN
BERGEPAINT.NS,674.0,11.51,-0.17,1.44,858.0,260.0,30.77,27.0,M,1.0,1.0,V40,NaN
DIXON.NS,4547.0,24.36,0.27,4.69,5750.0,374.0,22.38,26.0,M,1.0,1.0,V40N,NaN


In [26]:
# top 5 from star + v40/v40n for SIP
tmp_df.sort_values(by = 'Dev%_200', ascending=True).head(5)

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
RAJESHEXPO.NS,621.0,-7.38,0.00,-0.34,949.0,440.0,64.44,53.0,M,1.0,1.0,V40N,NaN
SUNTV.NS,445.0,-4.95,-0.21,0.92,755.0,249.0,61.26,70.0,M,1.0,1.0,V40N,NaN
ASTRAZEN.NS,3643.0,10.61,0.73,1.03,4650.0,1316.0,30.20,28.0,M,1.0,1.0,V40N,NaN
BERGEPAINT.NS,674.0,11.51,-0.17,1.44,858.0,260.0,30.77,27.0,M,1.0,1.0,V40,NaN
DIXON.NS,4547.0,24.36,0.27,4.69,5750.0,374.0,22.38,26.0,M,1.0,1.0,V40N,NaN


In [27]:
# star + v40/v40n stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
COLPAL.NS,1660.0,7.77,0.32,1.01,1719.0,931.0,7.49,4.0,H,1.0,1.0,V40,NaN
MARICO.NS,548.0,8.12,0.20,1.16,569.0,229.0,6.18,4.0,M,1.0,1.0,V40,NaN
BAJAJ-AUTO.NS,4642.0,18.60,0.87,1.91,4801.0,1803.0,5.30,3.0,L,1.0,1.0,V40,NaN
AKZOINDIA.NS,2452.0,9.63,1.07,0.46,2508.0,1360.0,4.88,2.0,M,1.0,1.0,V40,NaN
CERA.NS,7846.0,31.58,2.48,2.19,7978.0,2078.0,2.24,2.0,H,1.0,1.0,V40N,NaN
EQUITASBNK.NS,86.0,41.31,3.28,2.56,88.0,33.0,3.64,2.0,M,1.0,1.0,V40N,NaN
ICICIBANK.NS,934.0,4.50,0.45,0.76,954.0,252.0,2.85,2.0,H,1.0,1.0,V40,NaN
ABBOTINDIA.NS,22712.0,10.39,0.76,0.20,22892.0,6423.0,1.09,1.0,M,1.0,1.0,V40,NaN
POLYCAB.NS,3575.0,24.04,1.98,2.14,3595.0,526.0,0.65,1.0,M,1.0,1.0,V40N,NaN


In [28]:
# star stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 11


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
ADANITRANS.NS,811.0,-61.48,-6.60,-2.10,4105.0,147.0,83.22,406.0,L,1.0,1.0,NaN,NaN
ADANIGREEN.NS,975.0,-35.60,-4.49,0.64,2970.0,25.0,67.74,205.0,L,1.0,1.0,NaN,NaN
ADANIENT.NS,2510.0,-12.13,-0.98,3.65,4165.0,66.0,40.38,66.0,M,1.0,1.0,NaN,NaN
LTIM.NS,5011.0,8.83,0.13,0.64,7439.0,1243.0,39.19,48.0,M,1.0,1.0,V200,NaN
EMAMILTD.NS,403.0,-4.28,-0.95,0.79,588.0,144.0,41.67,46.0,L,1.0,1.0,V200,NaN
GODREJIND.NS,498.0,12.59,0.00,1.74,644.0,235.0,35.70,29.0,L,1.0,1.0,NaN,NaN
TATAPOWER.NS,222.0,4.83,0.00,1.46,285.0,26.0,24.32,28.0,M,1.0,1.0,NaN,NaN
SBICARD.NS,913.0,11.74,-0.12,2.37,1133.0,496.0,34.54,24.0,M,1.0,1.0,NaN,NaN
INDIANB.NS,282.0,8.86,1.93,0.00,346.0,39.0,20.85,23.0,H,1.0,1.0,NaN,NaN


In [29]:
# star stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
HINDUNILVR.NS,2716.0,5.50,0.16,0.66,2728.0,1406.0,0.91,0.0,H,1.0,1.0,V40,NaN
ITC.NS,453.0,24.82,1.93,1.42,453.0,124.0,0.00,0.0,H,1.0,1.0,V40,NaN
TITAN.NS,2954.0,13.98,0.81,1.47,2954.0,733.0,0.00,0.0,H,1.0,1.0,V40,NaN
ABBOTINDIA.NS,22712.0,10.39,0.76,0.20,22892.0,6423.0,1.09,1.0,M,1.0,1.0,V40,NaN
POLYCAB.NS,3575.0,24.04,1.98,2.14,3595.0,526.0,0.65,1.0,M,1.0,1.0,V40N,NaN
AKZOINDIA.NS,2452.0,9.63,1.07,0.46,2508.0,1360.0,4.88,2.0,M,1.0,1.0,V40,NaN
CERA.NS,7846.0,31.58,2.48,2.19,7978.0,2078.0,2.24,2.0,H,1.0,1.0,V40N,NaN
EQUITASBNK.NS,86.0,41.31,3.28,2.56,88.0,33.0,3.64,2.0,M,1.0,1.0,V40N,NaN
ICICIBANK.NS,934.0,4.50,0.45,0.76,954.0,252.0,2.85,2.0,H,1.0,1.0,V40,NaN


In [30]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
PGHH.NS,13947.0,0.51,-0.15,0.25,15778.0,8145.0,23.99,13.0,L,0.0,0.0,V40,NaN
NESTLEIND.NS,22969.0,15.93,0.75,1.64,22969.0,8621.0,0.00,0.0,H,0.0,0.0,V40,NaN


In [31]:
# stocks prospects

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 0


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,


In [32]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 14


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
JCHAC.NS,1072.0,-9.89,-1.85,0.09,3219.0,1017.0,97.50,200.0,M,1.0,0.0,V40N,NaN
LALPATHLAB.NS,2026.0,-7.13,-1.10,0.87,4186.0,807.0,63.92,107.0,M,1.0,0.0,V40N,NaN
WHIRLPOOL.NS,1468.0,0.11,-1.23,1.02,2667.0,1254.0,84.85,82.0,M,1.0,0.0,V40,NaN
NAM-INDIA.NS,246.0,-1.06,-1.20,1.68,438.0,118.0,60.00,78.0,L,1.0,0.0,V40,NaN
SUNTV.NS,445.0,-4.95,-0.21,0.92,755.0,249.0,61.26,70.0,M,1.0,1.0,V40N,NaN
SFL.NS,1188.0,-3.99,-1.46,1.89,1972.0,566.0,55.76,66.0,M,1.0,0.0,V40N,NaN
SYMPHONY.NS,915.0,-2.87,-0.11,-1.41,1470.0,707.0,72.74,61.0,M,1.0,0.0,V40N,NaN
ISEC.NS,526.0,6.58,0.40,1.86,829.0,164.0,45.56,58.0,M,1.0,0.0,V40N,NaN
TTKPRESTIG.NS,740.0,-9.42,-1.10,0.56,1165.0,391.0,54.91,57.0,M,1.0,0.0,V40N,NaN


In [33]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 7


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
JCHAC.NS,1072.0,-9.89,-1.85,0.09,3219.0,1017.0,97.50,200.0,M,1.0,0.0,V40N,NaN
WHIRLPOOL.NS,1468.0,0.11,-1.23,1.02,2667.0,1254.0,84.85,82.0,M,1.0,0.0,V40,NaN
SYMPHONY.NS,915.0,-2.87,-0.11,-1.41,1470.0,707.0,72.74,61.0,M,1.0,0.0,V40N,NaN
SIS.NS,414.0,5.83,-0.51,2.07,605.0,321.0,67.25,46.0,M,1.0,0.0,V40N,NaN
RAJESHEXPO.NS,621.0,-7.38,0.00,-0.34,949.0,440.0,64.44,53.0,M,1.0,1.0,V40N,NaN
LALPATHLAB.NS,2026.0,-7.13,-1.10,0.87,4186.0,807.0,63.92,107.0,M,1.0,0.0,V40N,NaN
SUNTV.NS,445.0,-4.95,-0.21,0.92,755.0,249.0,61.26,70.0,M,1.0,1.0,V40N,NaN


In [34]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 23


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
TITAN.NS,2954.0,13.98,0.81,1.47,2954.0,733.0,0.00,0.0,H,1.0,1.0,V40,NaN
NESTLEIND.NS,22969.0,15.93,0.75,1.64,22969.0,8621.0,0.00,0.0,H,0.0,0.0,V40,NaN
ITC.NS,453.0,24.82,1.93,1.42,453.0,124.0,0.00,0.0,H,1.0,1.0,V40,NaN
AXISBANK.NS,982.0,12.69,1.26,1.43,986.0,303.0,0.59,0.0,M,1.0,0.0,V40,NaN
POLYCAB.NS,3575.0,24.04,1.98,2.14,3595.0,526.0,0.65,1.0,M,1.0,1.0,V40N,NaN
HINDUNILVR.NS,2716.0,5.50,0.16,0.66,2728.0,1406.0,0.91,0.0,H,1.0,1.0,V40,NaN
ABBOTINDIA.NS,22712.0,10.39,0.76,0.20,22892.0,6423.0,1.09,1.0,M,1.0,1.0,V40,NaN
CERA.NS,7846.0,31.58,2.48,2.19,7978.0,2078.0,2.24,2.0,H,1.0,1.0,V40N,NaN
ICICIBANK.NS,934.0,4.50,0.45,0.76,954.0,252.0,2.85,2.0,H,1.0,1.0,V40,NaN


In [35]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 24


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
DIXON.NS,4547.0,24.36,0.27,4.69,5750.0,374.0,22.38,26.0,M,1.0,1.0,V40N,NaN
ANGELONE.NS,1556.0,17.25,0.83,3.07,1889.0,207.0,19.80,21.0,M,1.0,1.0,V40N,NaN
CAPLIPOINT.NS,812.0,14.56,0.14,2.97,947.0,190.0,17.83,17.0,M,1.0,1.0,V40N,NaN
SANOFI.NS,6930.0,24.16,0.95,2.50,8070.0,3818.0,26.81,16.0,M,0.0,0.0,V40,NaN
OFSS.NS,3870.0,27.68,0.96,2.21,4392.0,1232.0,16.52,13.0,M,1.0,1.0,V40N,NaN
POLYCAB.NS,3575.0,24.04,1.98,2.14,3595.0,526.0,0.65,1.0,M,1.0,1.0,V40N,NaN
BAJAJ-AUTO.NS,4642.0,18.60,0.87,1.91,4801.0,1803.0,5.30,3.0,L,1.0,1.0,V40,NaN
3MINDIA.NS,27931.0,20.92,1.16,1.90,29373.0,15332.0,10.27,5.0,M,1.0,1.0,V40N,NaN
EICHERMOT.NS,3539.0,5.14,0.36,1.86,3850.0,1252.0,11.97,9.0,H,1.0,1.0,V40N,ATH


In [36]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 20


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
TATAELXSI.NS,7640.0,8.70,-1.64,2.59,10685.0,529.0,29.98,40.0,M,0.0,0.0,V40N,NaN
SIS.NS,414.0,5.83,-0.51,2.07,605.0,321.0,67.25,46.0,M,1.0,0.0,V40N,NaN
ICICIPRULI.NS,534.0,13.42,-0.64,1.97,716.0,238.0,38.08,34.0,M,1.0,0.0,V40,NaN
SFL.NS,1188.0,-3.99,-1.46,1.89,1972.0,566.0,55.76,66.0,M,1.0,0.0,V40N,NaN
NAM-INDIA.NS,246.0,-1.06,-1.20,1.68,438.0,118.0,60.00,78.0,L,1.0,0.0,V40,NaN
HONAUT.NS,42040.0,7.57,-0.11,1.51,48839.0,17924.0,21.99,16.0,M,1.0,0.0,V40N,NaN
KANSAINER.NS,453.0,5.29,-0.70,1.48,659.0,292.0,56.13,45.0,M,1.0,0.0,V40N,NaN
BERGEPAINT.NS,674.0,11.51,-0.17,1.44,858.0,260.0,30.77,27.0,M,1.0,1.0,V40,NaN
ERIS.NS,660.0,0.69,-0.15,1.44,828.0,352.0,35.29,25.0,M,1.0,0.0,V40N,NaN


In [37]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] != 1) | (tmp_df['BizDurability'] != 1)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 14


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
JCHAC.NS,1072.0,-9.89,-1.85,0.09,3219.0,1017.0,97.50,200.0,M,1.0,0.0,V40N,NaN
LALPATHLAB.NS,2026.0,-7.13,-1.10,0.87,4186.0,807.0,63.92,107.0,M,1.0,0.0,V40N,NaN
WHIRLPOOL.NS,1468.0,0.11,-1.23,1.02,2667.0,1254.0,84.85,82.0,M,1.0,0.0,V40,NaN
NAM-INDIA.NS,246.0,-1.06,-1.20,1.68,438.0,118.0,60.00,78.0,L,1.0,0.0,V40,NaN
SUNTV.NS,445.0,-4.95,-0.21,0.92,755.0,249.0,61.26,70.0,M,1.0,1.0,V40N,NaN
SFL.NS,1188.0,-3.99,-1.46,1.89,1972.0,566.0,55.76,66.0,M,1.0,0.0,V40N,NaN
SYMPHONY.NS,915.0,-2.87,-0.11,-1.41,1470.0,707.0,72.74,61.0,M,1.0,0.0,V40N,NaN
ISEC.NS,526.0,6.58,0.40,1.86,829.0,164.0,45.56,58.0,M,1.0,0.0,V40N,NaN
TTKPRESTIG.NS,740.0,-9.42,-1.10,0.56,1165.0,391.0,54.91,57.0,M,1.0,0.0,V40N,NaN


In [38]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
PVRINOX.NS,1451.0,-10.21,-0.74,-0.41,1917.0,1348.0,81.9,32.0,L,0.0,0.0,NaN,NaN


In [39]:
# save the output
df_prec_dev = df_prec_dev.reset_index()
df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [40]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BHARTIARTL.NS')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-05-19 00:00:00+05:30,806.0,BHARTIARTL.NS,3.30,0.77,0.52,851.0,258.0,7.59,6.0
2023-05-26 00:00:00+05:30,818.0,BHARTIARTL.NS,4.46,0.77,0.52,851.0,258.0,5.56,4.0
2023-06-02 00:00:00+05:30,836.0,BHARTIARTL.NS,6.29,0.89,0.89,851.0,258.0,2.53,2.0
2023-06-09 00:00:00+05:30,834.0,BHARTIARTL.NS,5.51,0.89,1.01,851.0,258.0,2.87,2.0
2023-06-16 00:00:00+05:30,838.0,BHARTIARTL.NS,5.78,0.63,1.00,851.0,258.0,2.19,2.0
